In [1]:
#Run this cell to load dependencies
drive_path = '/Volumes/Brain2016'
import numpy as np
import pandas as pd
import os
import h5py
import glob
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
manifest_path = os.path.join(drive_path,'BrainObservatory','manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_path)

In [9]:
#Run this cell to define the function for use, YOU MUST have Saskia's analysis files are in the file_path below
def get_good_cell_ids(A=True,B=True,C=True,andor='and',cutoff=1.):
    '''
    This function goes through the analysis output data provided by Saskia and selects cells with mean responses
    to drifting/static gratings > cutoff%, removes suppressed by contrast cells, then takes the union or intersection
    of the cell & experiment ids depending on andor input
    
    '''
    file_path = r'/Volumes/Brain2016/BrainObservatory/ophys_analysis/*.h5'
    file_list = glob.glob(file_path)
    good_A = []
    good_B = []
    good_C = []
    exp_id_list = []
    cnt=0
    for file in file_list:
        f = h5py.File(file)
        exp_id = file[-37:-28]
        data_set = boc.get_ophys_experiment_data(int(exp_id))
        csids = data_set.get_cell_specimen_ids()
        exp_id_list = np.hstack((exp_id_list,exp_id))
        if 'session_A' in file:
            response = f["analysis/response_dg"].value
            f.close()
            cells = csids[(response[:,1:,:-1,0].max(axis=(0,1))>cutoff)>response[0,0,:-1,0]]
            good_A = np.hstack((good_A,cells))
        elif 'session_B' in file:
            peak = pd.read_hdf(file, 'analysis/peak')
            cells = csids[(peak.peak_dff_sg>cutoff).values]
            good_B = np.hstack((good_B,cells))
            f.close()
        else:
            f.close()
            cells = csids
            good_C = np.hstack((good_C,cells))

    if andor=='and':
        if A and B and not C:
            good_cell_ids = np.intersect1d(good_A,good_B)
            return good_cell_ids,exp_id_list
        elif A and not B and C:
            good_cell_ids = np.intersect1d(good_A,good_C)
            return good_cell_ids,exp_id_list
        elif not A and B and C:
            good_cell_ids = np.intersect1d(good_B,good_C)
            return good_cell_ids,exp_id_list
        elif A and B and C:
            good_cell_ids = np.intersect1d(good_A,np.intersect1d(good_B,good_C))
            return good_cell_ids,exp_id_list
        elif A and not B and not C:
            good_cell_ids = good_A
            return good_cell_ids,exp_id_list
        elif B and not A and not C:
            good_cell_ids = good_A
            return good_cell_ids,exp_id_list
        elif C and not A and not B:
            good_cell_ids = good_A
            return good_cell_ids,exp_id_list
        else:
            raise NameError('You need to choose True for at least one session')
    elif andor=='or':
        if A and B and not C:
            good_cell_ids = np.union1d(good_A,good_B)
            return good_cell_ids,exp_id_list
        elif A and not B and C:
            good_cell_ids = np.union1d(good_A,good_C)
            return good_cell_ids,exp_id_list
        elif not A and B and C:
            good_cell_ids = np.union1d(good_B,good_C)
            return good_cell_ids,exp_id_list
        elif A and B and C:
            good_cell_ids = np.union1d(good_A,np.union1d(good_B,good_C))
            return good_cell_ids,exp_id_list
        elif A and not B and not C:
            good_cell_ids = good_A
            return good_cell_ids,exp_id_list
        elif B and not A and not C:
            good_cell_ids = good_A
            return good_cell_ids,exp_id_list
        elif C and not A and not B:
            good_cell_ids = good_A
            return good_cell_ids,exp_id_list
        else:
            raise NameError('You need to choose True for at least one session')
        

    print("%d good drifting gratings cells"%len(good_dgs))
    print("%d good static gratings cells"%len(good_sgs))
    print("%d good cells for both stimuli"%len(good_cell_ids))

In [10]:
#Pull out the cells IDs and experiment IDs based on your criteria!
'''
Set each session markers (A, B, and C) to True/False to get cell ids from those sessions
Set andor to 'and' to get cells with data in all selected sessions, 'or' to get any cells from selected sessions
Set cutoff to the percent dfof value for thresholding (e.g. use 1. for 1% dfof)

example: get_good_cell_ids(A=True,B=True,C=True,andor='or',cutoff=1.)
    gives you cell and experiment ids for cells with data in all three sessions
    that have >1% dfof response to preferred stimulus

'''
ids,exps = get_good_cell_ids(A=True,B=True,C=False,andor='and',cutoff=-1000)
print('%d cells in %d experiments'%(len(ids),len(exps)))

5716 cells in 237 experiments


In [11]:
np.save('noSBC_cell_ids.npy',ids)